In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm
from dataset import get_datasets
from model import get_model
from kornia import augmentation as K
from kornia.augmentation import AugmentationSequential
import time
import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

c:\Users\doryo\Desktop\deep_learning\Project\convnext.py:158: UserWarning: Overwriting convnext_tiny in registry with convnext.convnext_tiny. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  @register_model
c:\Users\doryo\Desktop\deep_learning\Project\convnext.py:167: UserWarning: Overwriting convnext_small in registry with convnext.convnext_small. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  @register_model
c:\Users\doryo\Desktop\deep_learning\Project\convnext.py:176: UserWarning: Overwriting convnext_base in registry with convnext.convnext_base. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  @register_model
c:\Users\doryo\Desktop\deep_learning\Project\convnext.py:185: UserWarning: Overwriting convnext_large in registry with convnext.convnext_large. This is because the name being regi

In [2]:
image_dims = (224, 224)

transform = transforms.Compose([
    transforms.Resize(image_dims),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4772, 0.4597, 0.4612], std=[0.2997, 0.2808, 0.2837])
])

train_dataset, val_dataset, _ = get_datasets(transform)

In [3]:
# model
model = get_model().to(device)

# hyper parameters from optuna
batch_size = 32
lr_slow = 0.00014
weight_decay_slow = 0.00113
lr_fast = 0.00942
weight_decay_fast = 0.00183

optimizer_slow = torch.optim.Adam([{'params':model.stages[3].parameters()},
                                    {'params':model.norm.parameters()}], lr=lr_slow , weight_decay=weight_decay_slow)
optimizer_fast = torch.optim.Adam(model.head.parameters(), lr=lr_fast, weight_decay=weight_decay_fast)

# hyper parameters not from optuna
epochs = 200
criterion = nn.CrossEntropyLoss()
scheduler_slow = torch.optim.lr_scheduler.MultiStepLR(optimizer_slow, milestones=[75, 150], gamma=0.1)
scheduler_fast = torch.optim.lr_scheduler.MultiStepLR(optimizer_fast, milestones=[75, 150], gamma=0.1)


In [4]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [5]:
# Augmentations

aug_list = AugmentationSequential(
    K.ColorJitter(0.1, 0.1, 0.1, 0.1, p=0.9),
    K.RandomHorizontalFlip(p=0.5),
    K.RandomAffine(degrees=15.0, translate=(0.1, 0.1), scale=(0.8, 1.2), shear=(10.0, 10.0), p=0.5),
    K.RandomGaussianNoise(std=0.05, p=0.9),
    K.RandomCrop(size=(224, 224), padding=None, p=0.2),
    same_on_batch=False
)

In [6]:
# function to calcualte accuracy of the model
def calculate_accuracy(model, dataloader):
    model.eval()
    
    correct = 0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            pred = outputs.argmax(dim=1, keepdim=True)
            correct += pred.eq(labels.view_as(pred)).sum().item()

    accuracy = (correct / len(dataloader.dataset)) * 100

    return accuracy


In [7]:
loss_history = []
train_accuracy_history = []
val_accuracy_history = []

max_val_accurecy = 0.0

print('==> Start Training ...')
for epoch in tqdm(range(1, epochs + 1)):
    model.train()
    running_loss = 0.0
    epoch_time = time.time()
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        images = aug_list(images)
        labels = labels.to(device)

        output = model(images)
        loss = criterion(output, labels)
        optimizer_slow.zero_grad()
        optimizer_fast.zero_grad()
        loss.backward()
        optimizer_slow.step()
        optimizer_fast.step()

        running_loss += loss.data.item()

    # step the scheduler
    scheduler_slow.step()
    scheduler_fast.step()

    # Normalizing the loss by the total number of train batches
    running_loss /= len(train_loader)
      
    # Validation of the model.
    train_accuracy = calculate_accuracy(model, train_loader)
    val_accuracy = calculate_accuracy(model, val_loader)

    # save in history lists
    loss_history.append(running_loss)
    train_accuracy_history.append(train_accuracy)
    val_accuracy_history.append(val_accuracy)

    # log
    log = "Epoch: {} | Loss: {:.4f} | Training accuracy: {:.3f}% | Val accuracy: {:.3f}% | ".format(epoch, running_loss, train_accuracy, val_accuracy)
    epoch_time = time.time() - epoch_time
    log += "Epoch Time: {:.2f} secs".format(epoch_time)
    print(log)


    # save model
    if val_accuracy >= max_val_accurecy:
        max_val_accurecy = val_accuracy

        print('==> Saving model ...')
        state = {
            'model': model.state_dict(),
            'optimizer_fast': optimizer_fast.state_dict(),
            'optimizer_slow': optimizer_slow.state_dict(),
            'scheduler_fast': scheduler_fast.state_dict(),
            'scheduler_slow': scheduler_slow.state_dict(),
            'epoch': epoch,
            'loss_history': loss_history,
            'training_accuracy_history': train_accuracy_history,
            'val_accuracy_history': val_accuracy_history,
        }

        if not os.path.isdir('checkpoints'):
            os.mkdir('checkpoints')
        torch.save(state, f'./checkpoints/vehicle_damage_convnext_{epoch}.pth')


print('==> Finished Training ...')

==> Start Training ...


  0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 1 | Loss: 0.7785 | Training accuracy: 82.428% | Val accuracy: 80.140% | Epoch Time: 195.69 secs
==> Saving model ...


  0%|          | 1/200 [03:15<10:49:47, 195.92s/it]

Epoch: 2 | Loss: 0.4270 | Training accuracy: 87.801% | Val accuracy: 82.347% | Epoch Time: 179.71 secs
==> Saving model ...


...

Epoch: 198 | Loss: 0.0035 | Training accuracy: 99.885% | Val accuracy: 97.492% | Epoch Time: 184.34 secs


100%|█████████▉| 199/200 [10:48:56<03:04, 184.82s/it]

Epoch: 199 | Loss: 0.0021 | Training accuracy: 99.924% | Val accuracy: 97.693% | Epoch Time: 183.85 secs


100%|██████████| 200/200 [10:52:02<00:00, 195.61s/it]

Epoch: 200 | Loss: 0.0011 | Training accuracy: 99.924% | Val accuracy: 97.593% | Epoch Time: 186.56 secs
==> Finished Training ...


In [8]:
print('==> Saving model ...')
state = {
    'model': model.state_dict(),
    'optimizer_fast': optimizer_fast.state_dict(),
    'optimizer_slow': optimizer_slow.state_dict(),
    'scheduler_fast': scheduler_fast.state_dict(),
    'scheduler_slow': scheduler_slow.state_dict(),
    'epoch': epoch,
    'loss_history': loss_history,
    'training_accuracy_history': train_accuracy_history,
    'val_accuracy_history': val_accuracy_history,
}

if not os.path.isdir('checkpoints'):
    os.mkdir('checkpoints')
torch.save(state, f'./checkpoints/vehicle_damage_convnext_{epoch}.pth')

==> Saving model ...
